In [ ]:
from time import sleep

import gymnasium as gym
from ipycanvas import Canvas

env = gym.make('CartPole-v1',render_mode='rgb_array')
env.reset()

canvas = Canvas(width=400, height=600)
display(canvas)

for j in range(200):
    _, _, terminated, _, _ = env.step(j%2)
    canvas.put_image_data(env.render(), 0, 0)
    sleep(0.01)

In [ ]:
from math import pi

from ipywidgets import interact

from example_robot_data import load
from pinocchio.visualize import MeshcatVisualizer

In [ ]:
talos = load("talos")
viz = MeshcatVisualizer(talos.model, talos.collision_model, talos.visual_model)
viz.initViewer(loadModel=True)
viz.viewer.window.web_url = viz.viewer.window.web_url.replace("127.0.0.1", "hako.laas.fr")
viz.viewer.jupyter_cell()

In [ ]:
q0 = talos.q0
viz.display(q0)

In [ ]:
@interact(shoulder=(-0.5, 0.5, 0.01))
def say_hi(shoulder=0):
    q0[30] = pi - shoulder
    viz.display(q0)